In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Configuração dos gráficos
sns.set(style="whitegrid")


In [ ]:
# Carregar os dados (modifique o caminho conforme necessário)
df = pd.read_csv('base_exemplo_compras.csv', parse_dates=['purchase_date'])
df.head()


In [ ]:
# Calcular a data de referência
reference_date = df['purchase_date'].max() + pd.Timedelta(days=1)

# Calcular RFM
rfm = df.groupby('customer_id').agg({
    'purchase_date': lambda x: (reference_date - x.max()).days,
    'customer_id': 'count',
    'purchase_value': 'sum'
}).rename(columns={
    'purchase_date': 'Recency',
    'customer_id': 'Frequency',
    'purchase_value': 'Monetary'
}).reset_index()

rfm.head()


In [ ]:
# Padronizar os dados
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])


In [ ]:
# Clusterização com KMeans
kmeans = KMeans(n_clusters=4, random_state=42, n_init='auto')
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

rfm.head()


In [ ]:
# Visualização dos clusters
sns.pairplot(rfm, hue='Cluster', palette='tab10')
plt.suptitle("Segmentação de Clientes por RFM", fontsize=16)
plt.show()


In [ ]:
# Estatísticas por cluster
rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean().round(2)
